In [1]:
import time
import os
from google import genai
import pandas as pd
from google.genai import types
file_path = 'Dataset/1_label/V1_5 divisions_per_second.xlsx'
sheet_name = 'Metadata'

df = pd.read_excel(file_path, sheet_name=sheet_name)
df.set_index('Column (Header)', inplace=True)

gauge_type = df.loc['type', 'Content / Example']
unit = df.loc['unit', 'Content / Example']
reading_start = df.loc['manufacturer_range_min', 'Content / Example']
reading_end = df.loc['manufacturer_range_max', 'Content / Example']

reading_start = float(reading_start)
reading_end = float(reading_end)
graduation_interval = df.loc['graduation_interval', 'Content / Example']

client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])
# model_name = "gemini-3-flash-preview"
model_name = "gemini-3-pro-preview"


VIDEO_DIR = 'Dataset/1.- Analog Dial Gauge/'
video_files = [f for f in os.listdir(VIDEO_DIR) if f.endswith('.mp4')]
output_dir = f'Results_new/1.- Analog Dial Gauge{model_name}'
os.makedirs(output_dir, exist_ok=True)
    
for video_filename in video_files:
    seq_id = os.path.splitext(video_filename)[0]
    
    output_file = os.path.join(output_dir, f'{seq_id}_Raw_Results.xlsx')
    if os.path.exists(output_file):
        print(f"Skipping: {seq_id} (the result file already exists)")
        continue
    
    print(f"\n{'='*20} Processing: {seq_id} {'='*20}")

    print("Uploading...")
    program_start_time = time.time()
    myfile = client.files.upload(file=f"Dataset/1.- Analog Dial Gauge/{seq_id}.mp4")
    print(f"Uploaded: {myfile.name}")

    while True:
        file_status = client.files.get(name=myfile.name)

        if file_status.state == "ACTIVE":
            print("Succeed")
            break
        elif file_status.state == "FAILED":
            raise Exception("Failed (State: FAILED)")

        print("Still processing, waiting...")
        time.sleep(5)

        
# CoT prompt
    message = f'''
    ROLE: Expert Industrial Metrology Assistant

    PROTOCOL: See-Think-Confirm
    1. SEE: Localize gauge boundaries. Identify [0%] and [100%] markers.
    2. THINK: Synchronize needle position with the in-band digital clock. Calculate velocity (ΔReading/ΔTime). Use 'Thought Signatures' to maintain temporal state between frames.
    3. CONFIRM: Verify that the reading is within [min, max] and follows physical monotonicity relative to previous frames.
    CONSTRAINT: Output JSON only. Format: {{"ts_ms": integer, "reading": float, "confidence": float}}
    TASK: Extract a high-precision time-series of readings from the video.

    INSTRUMENT METADATA:
    - Gauge Type:{gauge_type} | Unit: {unit} 
    - Graduation Interval: {graduation_interval}
    - Calibrated Range: [{reading_start}, {reading_end}]

    SAMPLING PROTOCOL:
    - Absolute Time Reference: Every 'ts_ms' in your output must correspond to the EXACT numerical value shown on the digital chronometer in the frame.
    - Starting Point: The chronometer in this video starts at a non-zero value. Ignore any frames before the timer reaches the first 200ms integer increment (e.g., if the timer starts at 5670ms, your first entry should be at 5800ms).
    - Sampling Interval: Provide one reading every 200ms based on the chronometer's increments (e.g., 5800, 6000, 6200...). 
    - Duration: Continue this sequence until the video ends, strictly following the chronometer's values, regardless of the video file's elapsed time.

    RESPONSE REQUIREMENT:
    Output a single JSON array where each entry contains: {{"ts_ms": integer, "reading": float, "confidence": float}}
    '''

# naive prompt
#     message = f'''
#     ROLE: Expert Industrial Metrology Assistant

#     TASK: Read the analog gauge values from the video at specific time intervals.

#     INSTRUMENT METADATA:
#     - Gauge Type:{gauge_type} | Unit: {unit} 
#     - Graduation Interval: {graduation_interval}
#     - Calibrated Range: [{reading_start}, {reading_end}]

#     SAMPLING REQUIREMENTS:
#     - Use the digital chronometer shown in the video for timing.
#     - Start reading from the first 200ms integer increment.
#     - Provide one reading every 200ms (e.g., 5800, 6000, 6200...).
#     - Continue until the video ends.

#     OUTPUT FORMAT:
#     Return only a JSON array of objects: {{"ts_ms": integer, "reading": float, "confidence": float}}
#     '''
    print("Generatng...")

    response = client.models.generate_content(
        model=model_name, 
        contents=[myfile, message],
        config=types.GenerateContentConfig(
            temperature=0.0,
            thinking_config=types.ThinkingConfig(thinking_level="high")
        )
    )
    # "Identify the analog gauge and output a timestamped table of its readings based on the visible dial markings."
    print(response.text)


    program_end_time = time.time()
    total_duration = program_end_time - program_start_time
    print(total_duration)

    df_output = pd.DataFrame([{
        "sequence_id": seq_id,
        "raw_model_response": response.text,
        "total_duration_sec": round(total_duration, 2),
        "model_name": model_name,
    }])

    df_output.to_excel(output_file, index=False)


==================== Processing: V1_5 divisions_per_second ====================
Uploading...
Uploaded: files/67zsskpi2d6p
Still processing, waiting...
Still processing, waiting...
Succeed
Generatng...
```json
[
  {"ts_ms": 200, "reading": 4.85, "confidence": 1.0},
  {"ts_ms": 400, "reading": 4.85, "confidence": 1.0},
  {"ts_ms": 600, "reading": 4.85, "confidence": 1.0},
  {"ts_ms": 800, "reading": 4.85, "confidence": 1.0},
  {"ts_ms": 1000, "reading": 4.85, "confidence": 1.0},
  {"ts_ms": 1200, "reading": 4.85, "confidence": 1.0},
  {"ts_ms": 1400, "reading": 4.85, "confidence": 1.0},
  {"ts_ms": 1600, "reading": 4.85, "confidence": 1.0},
  {"ts_ms": 1800, "reading": 4.85, "confidence": 1.0},
  {"ts_ms": 2000, "reading": 4.85, "confidence": 1.0},
  {"ts_ms": 2200, "reading": 4.85, "confidence": 1.0},
  {"ts_ms": 2400, "reading": 4.85, "confidence": 1.0},
  {"ts_ms": 2600, "reading": 4.85, "confidence": 1.0},
  {"ts_ms": 2800, "reading": 4.85, "confidence": 1.0},
  {"ts_ms": 3000, "rea

Uploaded: files/4c46ie05ybhu
Still processing, waiting...
Still processing, waiting...
Succeed
Generatng...
[
  {"ts_ms": 2400, "reading": 5.97, "confidence": 0.95},
  {"ts_ms": 2600, "reading": 6.00, "confidence": 0.95},
  {"ts_ms": 2800, "reading": 6.03, "confidence": 0.95},
  {"ts_ms": 3000, "reading": 6.06, "confidence": 0.95},
  {"ts_ms": 3200, "reading": 6.09, "confidence": 0.95},
  {"ts_ms": 3400, "reading": 6.12, "confidence": 0.95},
  {"ts_ms": 3600, "reading": 6.15, "confidence": 0.95},
  {"ts_ms": 3800, "reading": 6.18, "confidence": 0.95},
  {"ts_ms": 4000, "reading": 6.21, "confidence": 0.95},
  {"ts_ms": 4200, "reading": 6.25, "confidence": 0.95},
  {"ts_ms": 4400, "reading": 6.28, "confidence": 0.95},
  {"ts_ms": 4600, "reading": 6.31, "confidence": 0.95},
  {"ts_ms": 4800, "reading": 6.34, "confidence": 0.95},
  {"ts_ms": 5000, "reading": 6.37, "confidence": 0.95},
  {"ts_ms": 5200, "reading": 6.40, "confidence": 0.95},
  {"ts_ms": 5400, "reading": 6.43, "confidence": 0

Uploaded: files/ov7jc771p8pd
Still processing, waiting...
Still processing, waiting...
Succeed
Generatng...
```json
[
  {"ts_ms": 2600, "reading": 4.92, "confidence": 0.95},
  {"ts_ms": 2800, "reading": 4.94, "confidence": 0.95},
  {"ts_ms": 3000, "reading": 4.96, "confidence": 0.95},
  {"ts_ms": 3200, "reading": 4.97, "confidence": 0.95},
  {"ts_ms": 3400, "reading": 4.99, "confidence": 0.95},
  {"ts_ms": 3600, "reading": 5.01, "confidence": 0.95},
  {"ts_ms": 3800, "reading": 5.03, "confidence": 0.95},
  {"ts_ms": 4000, "reading": 5.05, "confidence": 0.95},
  {"ts_ms": 4200, "reading": 5.07, "confidence": 0.95},
  {"ts_ms": 4400, "reading": 5.09, "confidence": 0.95},
  {"ts_ms": 4600, "reading": 5.11, "confidence": 0.95},
  {"ts_ms": 4800, "reading": 5.13, "confidence": 0.95},
  {"ts_ms": 5000, "reading": 5.15, "confidence": 0.95},
  {"ts_ms": 5200, "reading": 5.17, "confidence": 0.95},
  {"ts_ms": 5400, "reading": 5.19, "confidence": 0.95},
  {"ts_ms": 5600, "reading": 5.21, "confid

Uploaded: files/zyyodo4xg0lb
Still processing, waiting...
Succeed
Generatng...
```json
[
  {"ts_ms": 2200, "reading": 5.0, "confidence": 1.0},
  {"ts_ms": 2400, "reading": 5.0, "confidence": 1.0},
  {"ts_ms": 2600, "reading": 5.0, "confidence": 1.0},
  {"ts_ms": 2800, "reading": 5.0, "confidence": 1.0},
  {"ts_ms": 3000, "reading": 5.0, "confidence": 1.0},
  {"ts_ms": 3200, "reading": 5.0, "confidence": 1.0},
  {"ts_ms": 3400, "reading": 5.0, "confidence": 1.0},
  {"ts_ms": 3600, "reading": 5.0, "confidence": 1.0},
  {"ts_ms": 3800, "reading": 5.0, "confidence": 1.0},
  {"ts_ms": 4000, "reading": 5.0, "confidence": 1.0},
  {"ts_ms": 4200, "reading": 5.0, "confidence": 1.0},
  {"ts_ms": 4400, "reading": 5.0, "confidence": 1.0},
  {"ts_ms": 4600, "reading": 5.0, "confidence": 1.0},
  {"ts_ms": 4800, "reading": 5.0, "confidence": 1.0},
  {"ts_ms": 5000, "reading": 5.0, "confidence": 1.0},
  {"ts_ms": 5200, "reading": 5.0, "confidence": 1.0},
  {"ts_ms": 5400, "reading": 5.0, "confidence":

In [1]:
# test single frame
import time
import os
from google import genai
import pandas as pd
from google.genai import types
file_path = 'Dataset/1_label/V1_5 divisions_per_second.xlsx'
sheet_name = 'Metadata'

df = pd.read_excel(file_path, sheet_name=sheet_name)
df.set_index('Column (Header)', inplace=True)

gauge_type = df.loc['type', 'Content / Example']
unit = df.loc['unit', 'Content / Example']
reading_start = df.loc['manufacturer_range_min', 'Content / Example']
reading_end = df.loc['manufacturer_range_max', 'Content / Example']

reading_start = float(reading_start)
reading_end = float(reading_end)
client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])

VIDEO_DIR = 'Dataset/1.- Analog Dial Gauge/'
video_files = [f for f in os.listdir(VIDEO_DIR) if f.endswith('.mp4')]
# output_dir = 'Results/1.- Analog Dial Gauge'
# os.makedirs(output_dir, exist_ok=True)
target_timestamps = [9200]
video_files = ['V3_17 divisions_per_second.mp4']

for video_filename in video_files:
    seq_id = os.path.splitext(video_filename)[0]
    
    
    print(f"\n{'='*20} processing: {seq_id} {'='*20}")

    program_start_time = time.time()
    myfile = client.files.upload(file=f"Dataset/1.- Analog Dial Gauge/{seq_id}.mp4")


    while True:
        file_status = client.files.get(name=myfile.name)

        if file_status.state == "ACTIVE":
            break
        elif file_status.state == "FAILED":
            raise Exception("failed (State: FAILED)")

        print("waiting...")
        time.sleep(5)
    # -----------------------------

    # message = "Sample the video every 5 seconds. At each step, identify the time on the digital clock and detect ALL visible analog gauges. Read each gauge and present the data in a table with columns: [Timestamp, Digital Clock Time, Gauge Description, Inferred Scale, Reading]"
    # message = 'Read each gauge every 5 seconds and present the data in a table with columns: [Timestamp, Digital Clock Time, Gauge Description, Inferred Scale, Reading] where reading marks if the value increases as + or decreases as -'

    # message = '''Read each gauge every 5 seconds and present the data in a table with columns:
    # [Timestamp, Digital Clock Time, Gauge Description, Inferred Scale, Reading],
    # where Reading indicates whether the gauge value increases (+) or decreases (−) in the video.'''

    message = f'''
ROLE: Expert Industrial Metrology Assistant
PROTOCOL: See-Think-Confirm
1. SEE: Localize gauge boundaries. Identify [0%] and [100%] markers.
2. THINK: Synchronize needle position with the in-band digital clock. Calculate velocity (ΔReading/ΔTime). Use 'Thought Signatures' to maintain temporal state between frames.
3. CONFIRM: Verify that the reading is within [min, max] and follows physical monotonicity relative to previous frames.
CONSTRAINT: Output JSON only. Format: {{"ts_ms": integer, "stc_analysis": string, "reading": float, "conf": float}}
TASK: Extract a high-precision time-series of readings from the video.


INSTRUMENT METADATA:
- Gauge Type: {gauge_type} | Unit: {unit}

SAMPLING PROTOCOL:
- Target Timestamps (ts_ms): {target_timestamps}
- Requirement: For each timestamp in the list, locate the frame where the in-band digital chronometer matches this value.

RESPONSE REQUIREMENT:
Output a single JSON array where each entry contains: {{"ts_ms": integer, "stc_analysis": "SEE: ... | THINK: ... | CONFIRM: ...", "reading": float, "confidence": float}}
'''
    
#     model_name = "gemini-3-flash-preview"
#     model_name = "gemini-3-pro-preview"
    model_name = "gemini-3.1-pro-preview"
    response = client.models.generate_content(
        model=model_name, 
        contents=[myfile, message],
        config=types.GenerateContentConfig(
            temperature=0.0,  
            thinking_config=types.ThinkingConfig(thinking_level="high", include_thoughts=True)
        )
        
    )
    # "Identify the analog gauge and output a timestamped table of its readings based on the visible dial markings."
    print(response.text)
    
    for part in response.candidates[0].content.parts:
        if part and part.thought:  # show thoughts
            print(part.text)
        
    program_end_time = time.time()
    total_duration = program_end_time - program_start_time
    print(total_duration)



==================== processing: V3_17 divisions_per_second ====================
waiting...
waiting...
```json
[
  {
    "ts_ms": 9200,
    "stc_analysis": "SEE: Large dial needle is between 80 and 90. Small dial needle is between 0 and 1. | THINK: Small dial indicates 0mm. Large dial is at 84, which means 0.84mm. Total reading is 0.84mm. | CONFIRM: Reading is 0.84mm, consistent with the counter-clockwise movement.",
    "reading": 0.84,
    "conf": 0.95
  }
]
```
**Dial Gauge Reading at Timestamp 9200 ms**

Okay, so the task is to pinpoint a reading from a dial gauge at exactly 9200 milliseconds, or 00:09.20 on the chronometer. First, I need to find the frame corresponding to that specific time. Looking at the video data, I quickly locate the frame. The chronometer reads 00:09.20 at 9.2 seconds, which is what I'm after.

Now, let's analyze the dial gauge itself. I visually break it down: a large dial and a small one. The large dial, with its 0 to 100 scale, and major ticks at interva

In [2]:
response.candidates[0].content

Content(
  parts=[
    Part(
      text="""**Dial Gauge Reading at Timestamp 9200 ms**

Okay, so the task is to pinpoint a reading from a dial gauge at exactly 9200 milliseconds, or 00:09.20 on the chronometer. First, I need to find the frame corresponding to that specific time. Looking at the video data, I quickly locate the frame. The chronometer reads 00:09.20 at 9.2 seconds, which is what I'm after.

Now, let's analyze the dial gauge itself. I visually break it down: a large dial and a small one. The large dial, with its 0 to 100 scale, and major ticks at intervals of 10, clearly represents the fractional component of the measurement. Each minor tick is 0.01mm, so a full revolution is 1mm. The small dial indicates the whole number of millimeters, with each major tick representing 1mm and a range of 0 to 10.

At the target frame, my thought process goes like this: The small dial needle is clearly between 0 and 1, but close to zero, so the whole number part is definitely 0. Next, I l

In [3]:
content_parts = response.candidates[0].content.parts
thinking_process = content_parts[0].text if len(content_parts) > 0 else ""
print(thinking_process)

**Dial Gauge Reading at Timestamp 9200 ms**

Okay, so the task is to pinpoint a reading from a dial gauge at exactly 9200 milliseconds, or 00:09.20 on the chronometer. First, I need to find the frame corresponding to that specific time. Looking at the video data, I quickly locate the frame. The chronometer reads 00:09.20 at 9.2 seconds, which is what I'm after.

Now, let's analyze the dial gauge itself. I visually break it down: a large dial and a small one. The large dial, with its 0 to 100 scale, and major ticks at intervals of 10, clearly represents the fractional component of the measurement. Each minor tick is 0.01mm, so a full revolution is 1mm. The small dial indicates the whole number of millimeters, with each major tick representing 1mm and a range of 0 to 10.

At the target frame, my thought process goes like this: The small dial needle is clearly between 0 and 1, but close to zero, so the whole number part is definitely 0. Next, I look at the large dial. The needle is precis

In [1]:
import time
import os
import pandas as pd
file_path = 'Dataset/3_label/V1_5 divisions_per_second.xlsx'
sheet_name = 'Metadata'

df = pd.read_excel(file_path, sheet_name=sheet_name)
df.set_index('Column (Header)', inplace=True)

gauge_type = df.loc['type', 'Content / Example']
unit = df.loc['unit', 'Content / Example']
reading_start = df.loc['manufacturer_range_min', 'Content / Example']
reading_end = df.loc['manufacturer_range_max', 'Content / Example']

reading_start = float(reading_start)
reading_end = float(reading_end)
graduation_interval = df.loc['graduation_interval', 'Content / Example']



VIDEO_DIR = 'Dataset/1.- Analog Dial Gauge/'
video_files = [f for f in os.listdir(VIDEO_DIR) if f.endswith('.mp4')]

    
for video_filename in video_files:
    seq_id = os.path.splitext(video_filename)[0]
    
    
    print(f"\n{'='*20} Processing: {seq_id} {'='*20}")

    print("Uploading...")
    program_start_time = time.time()


        
# # CoT prompt
#     message = f'''
#     ROLE: Expert Industrial Metrology Assistant

#     PROTOCOL: See-Think-Confirm
#     1. SEE: Localize gauge boundaries. Identify [0%] and [100%] markers.
#     2. THINK: Synchronize needle position with the in-band digital clock. Calculate velocity (ΔReading/ΔTime). Use 'Thought Signatures' to maintain temporal state between frames.
#     3. CONFIRM: Verify that the reading is within [min, max] and follows physical monotonicity relative to previous frames.
#     CONSTRAINT: Output JSON only. Format: {{"ts_ms": integer, "reading": float, "confidence": float}}
#     TASK: Extract a high-precision time-series of readings from the video.

#     INSTRUMENT METADATA:
#     - Gauge Type:{gauge_type} | Unit: {unit} 
#     - Graduation Interval: {graduation_interval}
#     - Calibrated Range: [{reading_start}, {reading_end}]

#     SAMPLING PROTOCOL:
#     - Absolute Time Reference: Every 'ts_ms' in your output must correspond to the EXACT numerical value shown on the digital chronometer in the frame.
#     - Starting Point: The chronometer in this video starts at a non-zero value. Ignore any frames before the timer reaches the first 200ms integer increment (e.g., if the timer starts at 5670ms, your first entry should be at 5800ms).
#     - Sampling Interval: Provide one reading every 200ms based on the chronometer's increments (e.g., 5800, 6000, 6200...). 
#     - Duration: Continue this sequence until the video ends, strictly following the chronometer's values, regardless of the video file's elapsed time.

#     RESPONSE REQUIREMENT:
#     Output a single JSON array where each entry contains: {{"ts_ms": integer, "reading": float, "confidence": float}}
#     '''
# naive prompt
    message = f'''
    ROLE: Expert Industrial Metrology Assistant

    TASK: Read the analog gauge values from the video at specific time intervals.

    INSTRUMENT METADATA:
    - Gauge Type:{gauge_type} | Unit: {unit} 
    - Graduation Interval: {graduation_interval}
    - Calibrated Range: [{reading_start}, {reading_end}]

    SAMPLING REQUIREMENTS:
    - Use the digital chronometer shown in the video for timing.
    - Start reading from the first 200ms integer increment.
    - Provide one reading every 200ms (e.g., 5800, 6000, 6200...).
    - Continue until the video ends.

    OUTPUT FORMAT:
    Return only a JSON array of objects: {{"ts_ms": integer, "reading": float, "confidence": float}}
    '''
    print(message)
    break


==================== Processing: V1_5 divisions_per_second ====================
Uploading...

    ROLE: Expert Industrial Metrology Assistant

    TASK: Read the analog gauge values from the video at specific time intervals.

    INSTRUMENT METADATA:
    - Gauge Type:Caliper | Unit: mm 
    - Graduation Interval: 1
    - Calibrated Range: [0.0, 50.0]

    SAMPLING REQUIREMENTS:
    - Use the digital chronometer shown in the video for timing.
    - Start reading from the first 200ms integer increment.
    - Provide one reading every 200ms (e.g., 5800, 6000, 6200...).
    - Continue until the video ends.

    OUTPUT FORMAT:
    Return only a JSON array of objects: {"ts_ms": integer, "reading": float, "confidence": float}
    


In [ ]:
#     ROLE: Expert Industrial Metrology Assistant

#     PROTOCOL: See-Think-Confirm
#     1. SEE: Localize gauge boundaries. Identify [0%] and [100%] markers.
#     2. THINK: Synchronize needle position with the in-band digital clock. Calculate velocity (ΔReading/ΔTime). Use 'Thought Signatures' to maintain temporal state between frames.
#     3. CONFIRM: Verify that the reading is within [min, max] and follows physical monotonicity relative to previous frames.
#     CONSTRAINT: Output JSON only. Format: {{"ts_ms": integer, "reading": float, "confidence": float}}
#     TASK: Extract a high-precision time-series of readings from the video.

#     INSTRUMENT METADATA:
#     - Gauge Type:Circular | Unit: mm
#     - Graduation Interval: 0.01
#     - Calibrated Range: [0, 10]

#     SAMPLING PROTOCOL:
#     - Absolute Time Reference: Every 'ts_ms' in your output must correspond to the EXACT numerical value shown on the digital chronometer in the frame.
#     - Starting Point: The chronometer in this video starts at a non-zero value. Ignore any frames before the timer reaches the first 200ms integer increment (e.g., if the timer starts at 5670ms, your first entry should be at 5800ms).
#     - Sampling Interval: Provide one reading every 200ms based on the chronometer's increments (e.g., 5800, 6000, 6200...). 
#     - Duration: Continue this sequence until the video ends, strictly following the chronometer's values, regardless of the video file's elapsed time.

#     RESPONSE REQUIREMENT:
#     Output a single JSON array where each entry contains: {{"ts_ms": integer, "reading": float, "confidence": float}}

In [ ]:
# message = f'''
# ROLE: Expert Industrial Metrology Assistant.

# PROTOCOL: See-Think-Confirm (apply internally; output JSON only as specified below).

# 1. SEE
#    - Locate the analog gauge and its scale boundaries. Treat the scale minimum as [0%] and scale maximum as [100%] (these correspond to Calibrated Range below).
#    - Identify the digital chronometer in the frame and read its displayed time in milliseconds.
#    - Note needle position relative to tick marks; if the needle is between two marks, estimate by subdivision (use Graduation Interval).

# 2. THINK
#    - Map each required timestamp (ts_ms) to the correct moment in the video using the in-band digital chronometer as the sole time reference.
#    - If no frame shows the chronometer exactly at ts_ms: pick the nearest frame(s) and linearly interpolate the reading from adjacent chronometer values and needle positions (ΔReading/ΔTime). Maintain temporal continuity: each new reading should be consistent with the previous reading and the direction of motion (clockwise or counter-clockwise).
#    - Optionally use thought signatures or internal state to carry last_ts_ms, last_reading, and trend across frames.

# 3. CONFIRM
#    - Clamp every reading to the Calibrated Range [min, max].
#    - Enforce physical monotonicity: readings should not reverse direction unless the needle visibly reverses in the video. Small jitter from parallax or blur may be smoothed; if unsure, reflect it in confidence.
#    - Set confidence (0.0–1.0) by: visibility of scale and needle, alignment of chronometer with ts_ms, and consistency with neighboring readings. Lower confidence if interpolating heavily or if the gauge is partially occluded.

# CONSTRAINT: Output a single JSON array only. No markdown, no explanation outside the array. Each element: {{"ts_ms": integer, "reading": float, "confidence": float}}.

# TASK: Produce a high-precision time-series of gauge readings from the video at the specified sampling instants.

# INSTRUMENT METADATA:
# - Gauge Type: {gauge_type} | Unit: {unit}
# - Graduation Interval: {graduation_interval}
# - Calibrated Range: [{reading_start}, {reading_end}]

# SAMPLING PROTOCOL:
# - Time reference: Every output ts_ms must align with the digital chronometer value (in ms) visible in the video. Use the chronometer as the ground truth for time; ignore video file timestamps.
# - Start: Begin from the first 200 ms integer multiple reached by the chronometer (e.g. if it starts at 5670 ms, first output entry at 5800 ms). Omit any earlier frames.
# - Interval: One reading every 200 ms by chronometer (e.g. 5800, 6000, 6200, …).
# - End: Continue until the video ends, always using chronometer values for ts_ms.

# RESPONSE REQUIREMENT:
# Output only one JSON array. Each entry: {{"ts_ms": integer, "reading": float, "confidence": float}}. No other text before or after.
# '''